In [15]:
import numpy as np
import pandas as pd
import pandapower as pp

# Configurar opciones de impresión de NumPy y pandas
np.set_printoptions(precision=4, suppress=True, linewidth=100)
pd.options.display.float_format = '{:.4f}'.format

# Creación de una red eléctrica vacía
net = pp.create_empty_network()

# Crear barras
b1 = pp.create_bus(net, vn_kv=110, name="Barra 1")
b2 = pp.create_bus(net, vn_kv=220, name="Barra 2")
b1a = pp.create_bus(net, vn_kv=220, name="Barra 1A")
b2a = pp.create_bus(net, vn_kv=220, name="Barra 2A")
b3a = pp.create_bus(net, vn_kv=220, name="Barra 3A")
b1b = pp.create_bus(net, vn_kv=220, name="Barra 1B")
b2b = pp.create_bus(net, vn_kv=220, name="Barra 2B")

# Conexión de malla y transformador
pp.create_ext_grid(net, bus=b1, vm_pu=1, name="Conexion de malla")
pp.create_transformer(net, hv_bus=b2, lv_bus=b1, std_type="100 MVA 220/110 kV")

# Creación de líneas entre barras
pp.create_line(net, from_bus=b2, to_bus=b1a, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B2-B1A")
pp.create_line(net, from_bus=b2, to_bus=b1b, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B2-B1B")
pp.create_line(net, from_bus=b1a, to_bus=b2a, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B1A-B2A")
pp.create_line(net, from_bus=b2a, to_bus=b3a, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B2A-B3A")
pp.create_line(net, from_bus=b3a, to_bus=b2b, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B3A-B2B")
pp.create_line(net, from_bus=b2b, to_bus=b1b, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B2B-B1B")

# Valores nominales de carga y creación de cargas
P_nominal_MW = 150
Q_nominal_MVA = 100
bus_dict = {"b1a": pp.get_element_index(net, "bus", "Barra 1A"), "b2a": pp.get_element_index(net, "bus", "Barra 2A"),
            "b3a": pp.get_element_index(net, "bus", "Barra 3A"), "b2b": pp.get_element_index(net, "bus", "Barra 2B"),
            "b1b": pp.get_element_index(net, "bus", "Barra 1B")}
cargas = {"b1a": 0.20, "b2a": 0.35, "b3a": 0.15, "b2b": 0.60, "b1b": 0.10}
for barra, porcentaje in cargas.items():
    P = P_nominal_MW * porcentaje
    Q = Q_nominal_MVA * porcentaje
    pp.create_load(net, bus=bus_dict[barra], p_mw=P, q_mvar=Q, name=f"Carga en {barra}")

# Ejecución de la simulación de flujo de potencia
pp.runpp(net, max_iteration=4000, tolerance_mva=1e-9)

# Acceso y visualización de la matriz de admitancia como tabla
Ybus = net._ppc['internal']['Ybus']
Ybus_dense = np.array(Ybus.todense())
n_barras = 7  # Asegurarse de que este número coincida con el número de barras en la red
columnas = [f"Barra {i+1}" for i in range(n_barras)]  # Ajuste para que comience en 1
df_Ybus = pd.DataFrame(Ybus_dense, columns=columnas, index=columnas)
df_Ybus_complex = df_Ybus.applymap(lambda x: f"{x.real:+.4f}{x.imag:+.4f}j")
print("Matriz de Admitancia (Forma de Tabla):")
print(df_Ybus_complex)

# Imprimir resultados de simulación
print("\nResultados de las Barras:")
print(net.res_bus)

C:\Users\nachi\AppData\Local\Temp\ipykernel_30996\1488538898.py:54: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



Matriz de Admitancia (Forma de Tabla):
                    Barra 1                  Barra 2                  Barra 3  \
Barra 1  +18.0693-833.1437j       -18.0418+833.1317j          +0.0000+0.0000j   
Barra 2  -18.0418+833.1317j  +28090.8349-45050.0102j  -14036.3828+22117.9365j   
Barra 3     +0.0000+0.0000j  -14036.3828+22117.9365j  +23393.9714-36839.4693j   
Barra 4     +0.0000+0.0000j          +0.0000+0.0000j   -9357.5885+14745.2910j   
Barra 5     +0.0000+0.0000j          +0.0000+0.0000j          +0.0000+0.0000j   
Barra 6     +0.0000+0.0000j  -14036.3828+22117.9365j          +0.0000+0.0000j   
Barra 7     +0.0000+0.0000j          +0.0000+0.0000j          +0.0000+0.0000j   

                         Barra 4                  Barra 5  \
Barra 1          +0.0000+0.0000j          +0.0000+0.0000j   
Barra 2          +0.0000+0.0000j          +0.0000+0.0000j   
Barra 3   -9357.5885+14745.2910j          +0.0000+0.0000j   
Barra 4  +16375.7799-25770.9977j   -7018.1914+11058.9683j   
Barra 5